In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt, degrees, asin, floor

from skimage import data, color
from skimage.transform import hough_circle, probabilistic_hough_line, hough_circle_peaks
from skimage.feature import canny
from skimage.draw import circle_perimeter
from skimage.util import img_as_ubyte
from skimage.filters.edges import convolve
from numpy import array
from skimage.filters import roberts, sobel, scharr, prewitt
from skimage import img_as_float, img_as_ubyte
import cv2

font = {'family': 'arial',
        'color':  'green',
        'weight': 'normal',
        'size': 26}

perc=0
#Not used
def findEdges( image ):
    Kv = array([[ 1, 0,-1],
            [ 2, 0,-2],
            [ 1, 0,-1]]) 
    Kv = Kv * 0.5

    ver = np.abs(convolve(image, Kv))

    #     Progowanie
    MIN = np.percentile(sobel(ver), perc)
    MAX = np.percentile(sobel(ver), 100-perc)
    norm = (ver - MIN) / (MAX - MIN)
    norm[norm[:,:] > 1] = 1
    norm[norm[:,:] < 0.15] = 0
    
    return norm

def setFireArray(array, coordies):
    newArray = array
    maxLength = initMaxLength
    if len(array) == 0:
        newArray.append([coordies])
        return newArray
    found = False
    for index, group in enumerate(array):
        if found:
            break
        for indexInside, coordiesInside in enumerate(group): #[[[x1,y1],]]
            if (abs(coordiesInside[0] - coordies[0]) < maxLength and
                 abs(coordiesInside[1] - coordies[1]) < maxLength):
                found = True
                newArray[index].append(coordies)
                break
            else: 
                found = False
                
    if not found:
        newArray.append([coordies])
                
    return newArray

def getTime(startPixel, hand, radii):
    distance = sqrt(pow((hand[0] - startPixel[0]), 2) + pow((hand[1] - startPixel[1]), 2))
    print 'distance: ', distance

    if int(distance) == 2*radii[0]:
        return (0.5)

    angleSin = distance/(2*radii[0])
    print 'angleSin: ', angleSin

    print hand[1], startPixel[1]

    if angleSin > 1:
        angleSin = angleSin - 1

    angle = degrees(asin(angleSin))*2
    print 'angle: ', angle
    if hand[1] < startPixel[1]:
        angle = 360 - angle

    print 'changed angle: ', angle
    final = angle/360
    return final

print 'OK'

In [1476]:

# Ładując inny zmieniasz zmienną number
pwd = '/Users/marekmodkowski/Desktop/Polibud/KCK/ClockRecognizator/clocki/'
number = initNumber
image = img_as_float(data.load(pwd + 'clock_' + number + '.jpg', as_grey=True))
orgImage = image
edges = findEdges(image)
# algorytm canny - działa trochę gorzej ale szybciej
# edges = canny(image, sigma=3, low_threshold=10, high_threshold=50)
# fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(15, 6))
# fig2, ax2 = plt.subplots(ncols=1, nrows=1, figsize=(15, 6))

fig = plt.figure(figsize=(20, 8))
ax = fig.add_subplot(121)
ax2 = fig.add_subplot(122)

print 'OK'


OK


In [1477]:
# rozmiar okręgów - szuka okręgów o rozmiarze od 200 do 220  na obrazku z krawędziami
# Tak przynajmniej to rozumiem
hough_radii = np.arange(200, 220, 1) 
hough_res = hough_circle( edges, hough_radii)

# Classic straight-line Hough transform

# Określenie środków znalezionego koła cx - tablica x,
# cy - tablica y, radii - tablica z promieniami
# total_num_peaks - liczba okręgów jakie ma znaleźć
accums, cx, cy, radii = hough_circle_peaks(hough_res, hough_radii,
                                           total_num_peaks=1)
# Przy niektórych obrazkach przy rysowaniu wychodzi poza zakres, więc zmniejszam radii do 1
# radiiOrg - używam natomiast przy zoomowaniu
# Dzięki temu ryzsuje okrąg o rpromieniu 1 na środku znalezionego zegara
radiiOrg = radii
radii = [initRadii]
print 'OK'
image = color.gray2rgb(image)



OK


In [1478]:
# Rysowanie
r = 0
for center_y, center_x, radius in zip(cy, cx, radii):
    circy, circx = circle_perimeter(center_y, center_x, radius)
shapex, shapey, col = image.shape
print shapex, shapey
arrayOfFires = []
startPixel = [circy[0], circx[0]]
hourHand = []
minuteHand = []
minuteArray = []
hourArray = []

for  y, x in zip(circy[:len(circy)], circx[:len(circx)]):
#     print 'ORG ARR: : ',  y, x
    if y < startPixel[0]:
        startPixel = [y, x] 
    if x < shapex and y < shapey:
        if image[y, x].mean() < initMean:
                image[y, x] = [1, 1, 0]
                arrayOfFires = setFireArray(arrayOfFires, [y, x])
        else:
            image[y, x] = [0, 1, 1]
            

image[startPixel[0], startPixel[1]] = [1, 0, 0]
# print 'FINAL: ', arrayOfFires


for hand in arrayOfFires:
    if len(minuteArray) < len(hand):
        if len(hourArray) < len(hand):
            minuteArray = hourArray
            minuteHand = hourHand
            hourHand = hand[len(hand)/2]
            hourArray = hand
        else:
            minuteArray = hand
            minuteHand = hand[len(hand)/2]
            
#     print 'len: ', len(hand)
    mediumPixel = hand[len(hand)/2]
    image[mediumPixel[0], mediumPixel[1]] = [1, 0, 0]
    
image[hourHand[0], hourHand[1]] = [1, 0, 0]
image[minuteHand[0], minuteHand[1]] = [1, 0, 0]
print 'min len: ', len(minuteArray)
print 'hour len: ', len(hourArray)
# print 'min array: ', minuteHand
# print 'start pixel: ', startPixel

finalMinutes = int(getTime(startPixel, minuteHand, radii)*60)
if finalMinutes < 10:
    finalMinutesDisplayed = '0'+ str(finalMinutes)
else:
    finalMinutesDisplayed = str(finalMinutes)
    
finalHour = int(getTime(startPixel, hourHand, radii)*12)
if finalHour < 10:
    finalHourDisplayed = '0'+ str(finalHour)
else:
    finalHourDisplayed = str(finalHour)
    
finalTimeDisplayed = 'Time: ' + finalHourDisplayed + ':' + finalMinutesDisplayed

print 'finalTime: ', finalTimeDisplayed

# Zoom
zoomedImage = image[(cy[0]-radiiOrg[0]):(cy[0]+radiiOrg[0]), (cx[0]-radiiOrg[0]):(cx[0]+radiiOrg[0])]
zoomedEdges = edges[(cy[0]-radiiOrg[0]):(cy[0]+radiiOrg[0]), (cx[0]-radiiOrg[0]):(cx[0]+radiiOrg[0])]

ax2.imshow(edges, cmap=plt.cm.gray)
ax.imshow(zoomedImage, cmap=plt.cm.gray)


ax.text(0.5, 0.5, finalTimeDisplayed, verticalalignment='bottom', fontdict=font)
ax2.text(0.5, 0.5, 'Orginal: ', verticalalignment='bottom', fontdict=font)

plt.show()
fig.savefig('results_v04/result_clock_'+number+'.png')

450 450


IndexError: list index out of range

In [1479]:
initRadii = 55
initMaxLength = 10
initMean = 0.3
initNumber = '14'